<a href="https://colab.research.google.com/github/wpdias1/AED/blob/main/trabalho_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comandos para realização do trabalho da matéria de Big Data com uso da biblioteca PySpark.

Este notebook foi projetado para guiar os alunos na realização das práticas de Big Data utilizando PySpark. Certifique-se de seguir cada etapa cuidadosamente para garantir a correta execução das atividades.

Seu trabalho começará na célula 5. Execute as 4 primeiras células para iniciar a atividade.

## <font color=red>Observação importante:</font>

<font color=yellow>Trabalho realizado com uso da biblioteca pandas não será aceito!</font>

## Upload do arquivo `imdb-reviews-pt-br.csv` para dentro do Google Colab

Aqui, você fará o download do dataset necessário para as atividades. Certifique-se de que o arquivo foi descompactado corretamente antes de prosseguir.

In [73]:
!wget https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip -O imdb-reviews-pt-br.zip
!unzip imdb-reviews-pt-br.zip
!rm imdb-reviews-pt-br.zip

--2024-12-08 23:38:55--  https://raw.githubusercontent.com/N-CPUninter/Big_Data/main/data/imdb-reviews-pt-br.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49549692 (47M) [application/zip]
Saving to: ‘imdb-reviews-pt-br.zip’

imdb-reviews-pt-br. 100%[===================>]  47.25M   252MB/s    in 0.2s    

2024-12-08 23:38:55 (252 MB/s) - ‘imdb-reviews-pt-br.zip’ saved [49549692/49549692]

Archive:  imdb-reviews-pt-br.zip
replace imdb-reviews-pt-br.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: imdb-reviews-pt-br.csv  


## Instalação manual das dependências para uso do pyspark no Google Colab

Esta etapa garante que todas as bibliotecas necessárias para o PySpark sejam instaladas no Google Colab.

In [74]:
!pip install pyspark

## Importar, instanciar e criar a SparkSession

A SparkSession é o ponto de entrada para usar o PySpark. Certifique-se de configurar corretamente o nome do aplicativo e o master.

In [75]:
from pyspark.sql import SparkSession

appName = "PySpark Trabalho de Big Data"
master = "local"

spark = SparkSession.builder.appName(appName).master(master).getOrCreate()

## Criar spark dataframe do CSV utilizando o método read.csv do spark

Não altere este código e use o dataframe imdb_df criado aqui em todo o seu trabalho. A criação de um dataframe diferente deste poderá causar erros na coluna sentiment e isso refletirá em erros de resposta das questões.

In [76]:
imdb_df = spark.read.csv('imdb-reviews-pt-br.csv',
                         header=True,
                         quote="\"",
                         escape="\"",
                         encoding="UTF-8")

# Questão 1

Nesta questão, você irá calcular a soma dos IDs para entradas onde o sentimento ('sentiment') é 'neg'.

### Objetivo:
- Usar a coluna 'sentiment' como chave e somar os valores da coluna 'id'.

In [77]:
# Função MAP Transformar cada linha em tupla (chave, valor)
def map1(row):
    try:
        return (row['sentiment'], int(row['id']))
    except Exception as e:
        print(f"Erro ao processar linha: {row} -> {e}")
        return None

# Aplica a função MAP e filtra entradas válidas
mapped_rdd = imdb_df.rdd.map(map1).filter(lambda x: x is not None)

# Filtra apenas os casos onde 'sentiment' é igual a 'neg'
filtered_rdd = mapped_rdd.filter(lambda x: x[0] == 'neg')

# Visão inicial dos dados
imdb_df.show(5)

# Verificar dataFrame para confirmar as colunas
imdb_df.printSchema()



+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
+---+--------------------+--------------------+---------+
only showing top 5 rows

root
 |-- id: string (nullable = true)
 |-- text_en: string (nullable = true)
 |-- text_pt: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [78]:
def reduceByKey(x,y):
    # x e y são os valores (IDs) associados à mesma chave (sentiment)
    # A função simplesmente soma x e y
    return x + y

In [79]:
# Linha de código para aplicar o map/reduce no seu dataframe spark
result = filtered_rdd.reduceByKey(reduceByKey).collect()
# Coloque aqui o código para imprimir o resultado. Não esqueça seu RU:
print("Resultado da soma de IDs para 'neg':", result)
print("Minha RU: 4370725")

Resultado da soma de IDs para 'neg': [('neg', 459568555)]
Minha RU: 4370725


# Questão 2:

Nesta questão, você irá calcular a diferença no número total de palavras entre textos negativos em português e inglês.

### Objetivo:
- Contar as palavras em cada idioma (colunas 'text_pt' e 'text_en') para entradas onde o sentimento ('sentiment') é 'neg'.
- Subtrair o total de palavras em inglês do total em português.

## Criar funções de MAP:
- Criar função para mapear o "sentiment" como chave de uma tupla principal e como valor uma outra tupla com a soma das palavras de cada idioma como valor.

A função map irá transformar cada linha do dataframe em uma tupla (chave-valor), onde:
- Chave: coluna 'sentiment'
- Valor: Nova tupla com:
  - Elemento 0: soma das palavras da coluna 'text_en'
  - Elemento 1: soma das palavras da coluna 'text_pt'

OU
- Chave: coluna 'sentiment'
- Valor: (soma das palavras da coluna 'text_pt') - (soma das palavras da coluna 'text_en')
  

Para contar as palavras deve-se primeiro separar os textos em uma lista de palavras para então descobrir o tamanho desta lista.
Dicas:

1. Use o método .split() e não .split(" ") de string para separar as palavras em uma lista ou use a função split(coluna de texto, regex) do pyspark com o regex igual à "[ ]+" ou "\s+"
2. Use len() para descobrir o tamanho da lista de palavras.

In [80]:
# Função MAP transforma cada linha em uma tupla com contagem de palavras
def map2(row):
    try:
        sentiment = row['sentiment']
        if sentiment != 'neg':
            return None

        # Separa as palavras e calcula o total de cada idioma
        words_en = len(row['text_en'].split()) if row['text_en'] else 0
        words_pt = len(row['text_pt'].split()) if row['text_pt'] else 0

        # Retorna chave-valor para o RDD
        return (sentiment, (words_en, words_pt))
    except Exception as e:
        print(f"Erro ao processar linha: {row} -> {e}")
        return None

# Aplica função MAP e filtra entradas válidas
mapped_rdd = imdb_df.rdd.map(map2).filter(lambda x: x is not None)

# Visão inicial dos dados
imdb_df.show(5)

# Verifica esquema do DataFrame para confirmar as colunas
imdb_df.printSchema()

+---+--------------------+--------------------+---------+
| id|             text_en|             text_pt|sentiment|
+---+--------------------+--------------------+---------+
|  1|Once again Mr. Co...|Mais uma vez, o S...|      neg|
|  2|This is an exampl...|Este é um exemplo...|      neg|
|  3|First of all I ha...|Primeiro de tudo ...|      neg|
|  4|Not even the Beat...|Nem mesmo os Beat...|      neg|
|  5|Brass pictures mo...|Filmes de fotos d...|      neg|
+---+--------------------+--------------------+---------+
only showing top 5 rows

root
 |-- id: string (nullable = true)
 |-- text_en: string (nullable = true)
 |-- text_pt: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [81]:
# Função REDUCE (renomeada para reduceByKey2): Somar palavras por idioma
reduced_rdd = mapped_rdd.reduceByKey(
    lambda x, y: (x[0] + y[0], x[1] + y[1])  # x[0] e y[0] para inglês, x[1] e y[1] para português
)

In [82]:
# Coleta os resultados
result = reduced_rdd.collect()

# Calcula a diferença entre total de palavras em português e inglês
if result:
    total_en = result[0][1][0]  # Soma total de palavras em 'text_en'
    total_pt = result[0][1][1]  # Soma total de palavras em 'text_pt'
    difference = total_pt - total_en
# Exibindo em tela
    print(f"Total de palavras em inglês: {total_en}")
    print(f"Total de palavras em português: {total_pt}")
    print(f"Diferença (português - inglês): {difference}")
    print("Minha RU: 4370725")
else:
    print("Nenhum dado encontrado para 'sentiment' = 'neg'.")

# Finalizar a SparkSession
spark.stop()


Total de palavras em inglês: 5400324
Total de palavras em português: 5455273
Diferença (português - inglês): 54949
Minha RU: 4370725
